In [ ]:
import os
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

# os.environ["AWS_ACCESS_KEY_ID"] = "<aws-access-key-id>"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "<aws-secret-access-key>"
# os.environ["AWS_SESSION_TOKEN"] = "<aws-session-token>"

In [37]:
path = "/Users/sagarthacker/Downloads/jira-ticket-files/s3-211-incorporate-ai-understanding/20250813 Tracera Upload.csv"

df = pd.read_csv(path)

df.head()

,sku,Supplier,purchaseDate,countryOfOrigin,description1,description2,description3,productGrouping,Business Unit,spend,spendUnit,site,Supplier Number,Supplier Subsidiary
0,1.0,"Bengbu DOUBLECIRCLE ELECTRONICS GROUP CO.,LTD",3/31/25,Unknown,(blank),NaN,NaN,NaN,APAC,6946.56,2021 USD spend,YZ,D658L,"Bengbu DOUBLECIRCLE ELECTRONICS GROUP CO.,LTD"
1,2.0,GaoYou longqiu tangnan hardware factory,3/31/25,中国,printing,NaN,NaN,printing,APAC,17236.49,2021 USD spend,YZ,D333M,GaoYou longqiu tangnan hardware factory
2,3.0,"Nanjing NEDEKO electronics technology Co.,ltd",3/31/25,Unknown,printing,NaN,NaN,printing,APAC,379.83,2021 USD spend,YZ,D332M,"Nanjing NEDEKO electronics technology Co.,ltd"
3,4.0,"Wuxi Dezhili metal manufactures CO.,LTD",3/31/25,Unknown,MP STEEL TRAY,NaN,NaN,MP STEEL TRAY,APAC,529192.36,2021 USD spend,YZ,D105ST,"Wuxi Dezhili metal manufactures CO.,LTD"
4,5.0,'BEIJING RONGXINTONG DEVELOPME,3/31/25,Unknown,(blank),NaN,NaN,NaN,APAC,2284.73,2021 USD spend,CQ,GN103,'BEIJING RONGXINTONG DEVELOPME


In [38]:
# Drop rows where productGrouping is null
df = df.dropna(subset=["productGrouping"])

# Only keep the rows that have unique productGrouping, description1, description2, description3
df = df.drop_duplicates(subset=["productGrouping", "description1", "description2", "description3"])

# Fill na with "-" as the value
df = df.fillna("-")

df.head()

,sku,Supplier,purchaseDate,countryOfOrigin,description1,description2,description3,productGrouping,Business Unit,spend,spendUnit,site,Supplier Number,Supplier Subsidiary
1,2.0,GaoYou longqiu tangnan hardware factory,3/31/25,中国,printing,-,-,printing,APAC,17236.49,2021 USD spend,YZ,D333M,GaoYou longqiu tangnan hardware factory
3,4.0,"Wuxi Dezhili metal manufactures CO.,LTD",3/31/25,Unknown,MP STEEL TRAY,-,-,MP STEEL TRAY,APAC,529192.36,2021 USD spend,YZ,D105ST,"Wuxi Dezhili metal manufactures CO.,LTD"
10,11.0,***PAK-LITE INC,3/31/25,US,Other Direct Materials,-,-,Misc Other Direct,North America,37911.88,2021 USD spend,Atlanta,0000114448,***PAK-LITE INC
11,12.0,***POSDATA GROUP INC***,3/31/25,US,Printing,-,-,Labels,North America,13047.49,2021 USD spend,Atlanta,0000114084,***POSDATA GROUP INC***
13,14.0,*ACCUMA PLASTICS LIMITED,3/31/25,UK,Plastic,-,-,Injection Molding,North America,17526.09,2021 USD spend,Springfield A,AC0031,*ACCUMA PLASTICS LIMITED


In [39]:
# Read the prompt
with open("./core/prompts/product_understanding_prompt.txt", "r") as f:
    prompt = f.read()

# Replace the placeholders with the actual values
prompt = prompt.format(productGrouping="3rd party aluminum enclosures", description1="Aluminum", description2="-", description3="-")

print(prompt)

You are an AI assistant tasked with interpreting product information. Given the following input fields, provide a concise understanding of the product or service being described:

productGrouping: 3rd party aluminum enclosures
description1: Aluminum
description2: -
description3: -

Analyze the information provided and give a brief explanation of what you believe the product or service to be. Your response should be a direct statement of your understanding, without any introductory phrases.

Consider the following guidelines:

- Focus primarily on the productGrouping field, as it often contains the most relevant information.
- Use information from the description fields to provide additional context or details.
- If any field is marked with a dash (-), it means no information is provided for that field.
- Avoid making assumptions beyond what is directly implied by the given information.
- Keep your response concise, ideally within one or two sentences.

Examples:

Input: productGrouping

In [40]:
import sys
import os

# In a Jupyter notebook, __file__ is not defined. Use os.getcwd() instead.
notebook_dir = os.getcwd()
cocoon_root = os.path.abspath(os.path.join(notebook_dir, ".."))
if cocoon_root not in sys.path:
    sys.path.insert(0, cocoon_root)


In [44]:
# Load the bedrock llm 
from cocoon.core.llm.bedrock import BedrockLLM

llm = BedrockLLM(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", 
    region_name="eu-central-1", 
    model_kwargs={
        "temperature": 0.1
    })

In [103]:
# Load the prompt
with open("./core/prompts/product_understanding_prompt.txt", "r") as f:
    prompt = f.read()

# Replace the placeholders with the actual values
# prompt = prompt.format(productGrouping="Can", description1="CAN", description2="-", description3="-")
# prompt = prompt.format(productGrouping="Paint/powder coating", description1="Chemicals", description2="-", description3="-")
# prompt = prompt.format(productGrouping="Mro - bld maintenance", description1="Other MRO", description2="-", description3="-")
# prompt = prompt.format(productGrouping="Not classified", description1="Translation and interpretation services", description2="-", description3="-")
prompt = prompt.format(productGrouping="Misc separators", description1="Separators", description2="-", description3="-")
# prompt = prompt.format(productGrouping="sheet", description1="-", description2="-", description3="-")

In [104]:
res = llm.invoke([("human", prompt)])
print(res[0])

{
  "summary_search": "industrial separators, separation equipment, filtration devices, solid-liquid separation, particle separation, centrifugal separators, filter presses",
  "summary_display": "Manufacturing of industrial separation equipment used for solid-liquid separation, filtration, and particle separation processes.",
  "confidence": 0.75,
  "evidence_fields": ["productGrouping", "description1"]
}


In [89]:
# Loop through the csv file and run the prompt on each row

import pandas as pd

with open("./core/prompts/product_understanding_prompt.txt", "r") as f:
    prompt = f.read()

responses = []
for index, row in df.head().iterrows():
    updated_prompt = str(prompt)
    updated_prompt = prompt.format(productGrouping=row["productGrouping"], description1=row["description1"], description2=row["description2"], description3=row["description3"])
    
    response = llm.invoke(updated_prompt)
    product_input = f"Product Grouping: {row['productGrouping']}\nDescription 1: {row['description1']}\nDescription 2: {row['description2']}\nDescription 3: {row['description3']}"
    responses.append((product_input, response))

In [90]:
for product_input, response in responses:
    print(product_input)
    print(response[0])
    print("\n")

Product Grouping: printing 
Description 1: printing 
Description 2: -
Description 3: -
{
  "summary_search": "printing industry, commercial printing, print production, lithography, offset printing, digital printing, ink application, paper processing",
  "summary_display": "Commercial printing operations involving the application of ink to paper or other substrates using various printing technologies.",
  "confidence": 0.75,
  "evidence_fields": ["productGrouping", "description1"]
}


Product Grouping: MP STEEL TRAY
Description 1: MP STEEL TRAY
Description 2: -
Description 3: -
{
  "summary_search": "steel tray manufacturing, metal tray production, sheet metal forming, stamped steel products, industrial metal containers",
  "summary_display": "Manufacturing of steel trays through sheet metal forming and stamping processes for industrial or commercial use.",
  "confidence": 0.82,
  "evidence_fields": ["productGrouping", "description1"]
}


Product Grouping: Misc Other Direct
Description 

In [52]:
df.head()

,sku,Supplier,purchaseDate,countryOfOrigin,description1,description2,description3,productGrouping,Business Unit,spend,spendUnit,site,Supplier Number,Supplier Subsidiary
1,2.0,GaoYou longqiu tangnan hardware factory,3/31/25,中国,printing,-,-,printing,APAC,17236.49,2021 USD spend,YZ,D333M,GaoYou longqiu tangnan hardware factory
3,4.0,"Wuxi Dezhili metal manufactures CO.,LTD",3/31/25,Unknown,MP STEEL TRAY,-,-,MP STEEL TRAY,APAC,529192.36,2021 USD spend,YZ,D105ST,"Wuxi Dezhili metal manufactures CO.,LTD"
10,11.0,***PAK-LITE INC,3/31/25,US,Other Direct Materials,-,-,Misc Other Direct,North America,37911.88,2021 USD spend,Atlanta,0000114448,***PAK-LITE INC
11,12.0,***POSDATA GROUP INC***,3/31/25,US,Printing,-,-,Labels,North America,13047.49,2021 USD spend,Atlanta,0000114084,***POSDATA GROUP INC***
13,14.0,*ACCUMA PLASTICS LIMITED,3/31/25,UK,Plastic,-,-,Injection Molding,North America,17526.09,2021 USD spend,Springfield A,AC0031,*ACCUMA PLASTICS LIMITED
